In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.distributions.empirical_distribution import ECDF
from functools import partial

/Users/francis/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Data Handling
This notebook slices the data into training data and testing data

## Data
One single dataset from Bloomberg. 
- Daily closing price of BTC future and Daily Closing value of brr. 
- Data is merged by Bloomberg using the compare function


In [2]:
future_brr = pd.read_csv('../data/cleaned_data/BBT_Tiingo.csv')

In [3]:
future_brr

,Unnamed: 0,Date,PX_LAST,contract_name,BTC Price,log return future,log return spot
0,0,2021-05-27 20:00:00+00:00,38865.0,BTCM1 Curncy,38786.472240,0.006583,0.007532
1,2,2021-05-26 20:00:00+00:00,38610.0,BTCM1 Curncy,38495.443757,0.032643,0.031101
2,4,2021-05-25 20:00:00+00:00,37370.0,BTCM1 Curncy,37316.628068,-0.062245,-0.060677
3,6,2021-05-24 20:00:00+00:00,39770.0,BTCM1 Curncy,39650.983644,0.094330,0.093435
4,8,2021-05-21 20:00:00+00:00,36190.0,BTCM1 Curncy,36114.011225,-0.108809,-0.105785
...,...,...,...,...,...,...,...
749,1498,2018-06-08 20:00:00+00:00,7675.0,BTCN18 Curncy,7649.811567,-0.005846,-0.004447
750,1500,2018-06-07 20:00:00+00:00,7720.0,BTCN18 Curncy,7683.903049,0.022267,0.019947
751,1502,2018-06-06 20:00:00+00:00,7550.0,BTCN18 Curncy,7532.153431,-0.012504,-0.011713
752,1505,2018-06-05 20:00:00+00:00,7645.0,BTCN18 Curncy,7620.895741,0.017151,0.016024


## Slicing data

In [4]:
train_size = 300
test_size = 5
step_size = 5

In [5]:
len(future_brr)

754

In [8]:
data_name = 'BBT_Tiingo'

In [9]:
# n = int(np.floor((len(future_brr)-train_size)/step_size))
n = int(np.floor((len(future_brr) - train_size - test_size)/step_size))
train_set = []
test_set = []

for i in range(n):
    data = future_brr.iloc[(i*step_size):((i)*step_size)+train_size+test_size]
    test = data.iloc[:test_size,]
    train = data.iloc[test_size:,]
    train_set.append(train)
    test_set.append(test)

for i in range(len(train_set)):
    train_set[i].to_csv("../processed_data/%s/train/" %data_name +str(i)+".csv")
    test_set[i].to_csv("../processed_data/%s/test/" %data_name +str(i)+".csv")

In [10]:
test_set[0].head()

,Unnamed: 0,Date,PX_LAST,contract_name,BTC Price,log return future,log return spot
0,0,2021-05-27 20:00:00+00:00,38865.0,BTCM1 Curncy,38786.472240,0.006583,0.007532
1,2,2021-05-26 20:00:00+00:00,38610.0,BTCM1 Curncy,38495.443757,0.032643,0.031101
2,4,2021-05-25 20:00:00+00:00,37370.0,BTCM1 Curncy,37316.628068,-0.062245,-0.060677
3,6,2021-05-24 20:00:00+00:00,39770.0,BTCM1 Curncy,39650.983644,0.094330,0.093435
4,8,2021-05-21 20:00:00+00:00,36190.0,BTCM1 Curncy,36114.011225,-0.108809,-0.105785


In [11]:
train_set[0].head()

,Unnamed: 0,Date,PX_LAST,contract_name,BTC Price,log return future,log return spot
5,10,2021-05-20 20:00:00+00:00,40350.0,BTCM1 Curncy,40143.729949,0.021291,0.018658
6,12,2021-05-19 20:00:00+00:00,39500.0,BTCM1 Curncy,39401.672539,-0.090465,-0.093476
7,14,2021-05-18 20:00:00+00:00,43240.0,BTCM1 Curncy,43262.433661,-0.019694,-0.021612
8,16,2021-05-17 20:00:00+00:00,44100.0,BTCM1 Curncy,44207.612742,-0.131645,-0.126287
9,18,2021-05-14 20:00:00+00:00,50305.0,BTCM1 Curncy,50158.276955,0.036850,0.032827


In [12]:
train_set[0].tail()

,Unnamed: 0,Date,PX_LAST,contract_name,BTC Price,log return future,log return spot
300,600,2020-03-20 20:00:00+00:00,6230.0,BTCJ20 Curncy,6194.401518,-0.006400,-0.003845
301,602,2020-03-19 20:00:00+00:00,6270.0,BTCJ20 Curncy,6218.262575,0.159615,0.153577
302,604,2020-03-18 20:00:00+00:00,5345.0,BTCJ20 Curncy,5332.998328,-0.012088,-0.009687
303,606,2020-03-17 20:00:00+00:00,5410.0,BTCJ20 Curncy,5384.912732,0.089872,0.084526
304,608,2020-03-16 20:00:00+00:00,4945.0,BTCJ20 Curncy,4948.455895,-0.083382,-0.089169


In [13]:
test_set[1].head()

,Unnamed: 0,Date,PX_LAST,contract_name,BTC Price,log return future,log return spot
5,10,2021-05-20 20:00:00+00:00,40350.0,BTCM1 Curncy,40143.729949,0.021291,0.018658
6,12,2021-05-19 20:00:00+00:00,39500.0,BTCM1 Curncy,39401.672539,-0.090465,-0.093476
7,14,2021-05-18 20:00:00+00:00,43240.0,BTCM1 Curncy,43262.433661,-0.019694,-0.021612
8,16,2021-05-17 20:00:00+00:00,44100.0,BTCM1 Curncy,44207.612742,-0.131645,-0.126287
9,18,2021-05-14 20:00:00+00:00,50305.0,BTCM1 Curncy,50158.276955,0.036850,0.032827


In [14]:
train_set[-1].tail()

,Unnamed: 0,Date,PX_LAST,contract_name,BTC Price,log return future,log return spot
740,1480,2018-06-21 20:00:00+00:00,6730.0,BTCN18 Curncy,6713.804624,-0.005187,-0.005637
741,1482,2018-06-20 20:00:00+00:00,6765.0,BTCN18 Curncy,6751.755674,0.001479,0.001351
742,1484,2018-06-19 20:00:00+00:00,6755.0,BTCN18 Curncy,6742.643285,0.006684,0.006543
743,1486,2018-06-18 20:00:00+00:00,6710.0,BTCN18 Curncy,6698.671438,0.023371,0.022741
744,1488,2018-06-15 20:00:00+00:00,6555.0,BTCN18 Curncy,6548.053246,-0.014389,-0.014172
